## Informe de Extracción, Transformación y Carga de Datos (ETL): lesiones.xlsx

En este apartado, llevaremos a cabo el proceso de Extracción, Transformación y Carga (ETL) del archivo excel: "lesiones.xlsx", exploraremos cada hoja de trabajo (HECHOS y VICTIMAS) para lograr su comprensión y preparar los datos para una furutas exploraciones y análsis que puedan llevar hacia la acción de mitigar las pérdidas humanas en las carreteras de la Ciudad de Buenos Aires.

Empezaremos importando las bibliotecas necesarias para el trabajo, llevaremos a cabo técnicas de depuración y procesamiento, para finalmente almacenar los datos transformados en un archivo csv.

### 1. Importación de las bibliotecas necesarias

In [1]:
# Pandas para el análisis de datos tabulares
import pandas as pd

# NumPy proporciona soporte para arreglos y matrices multidimensionales
import numpy as np

# Seaborn para la visualización de datos mediante gráficos estadísticos
import seaborn as sns

# Matplotlib para la creación de gráficos y visualizaciones
import matplotlib.pyplot as plt

# Funciones útiles
from functions import cargar_archivos_excel
from functions import analizar_valores_sd
from functions import data_cleaning

### 2. Carga del archivo

In [2]:
# Ruta del Archivo
ruta = "../Datasets/lesiones.xlsx"

In [3]:
# Llamamos a la función "cargar_archivos_excel"
datos_lesiones = cargar_archivos_excel(ruta, ["HECHOS", "VICTIMAS"])

### 3. Exploración y Limpieza de Datos

#### 3.1. Dataset "HECHOS"

In [4]:
df_hechos = datos_lesiones["HECHOS"]

In [5]:
# Mostramos las primeras 3 filas del DataFrame
df_hechos.head(3)

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD


In [6]:
# Revisamos las información general del DataFrame
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

Tras verificar el DataFrame, identificamos que los valores "SD" corresponden a "Sin Dato", por lo que se tomó la decisión de cambiar estos valores a NaN para estandarizar.

In [9]:
# Observamos la presencia de datos "SD" en el DataFrame
df_hechos.eq("SD").sum()

id                           0
n_victimas                   0
aaaa                         0
mm                           0
dd                           0
fecha                        0
hora                         4
franja_hora                  0
direccion_normalizada    10815
comuna                     846
tipo_calle               11045
otra_direccion           18295
calle                        0
altura                       0
cruce                        0
geocodificacion_CABA      1213
longitud                  1209
latutid                   1209
victima                  10733
acusado                  15288
participantes                0
moto                      8511
auto                     12543
transporte_publico       11801
camion                   12708
ciclista                 11353
gravedad                 23056
dtype: int64

In [10]:
# Llamamos a la función "analizar_valores_sd"
analizar_valores_sd(df_hechos)

,Columna,Cantidad de SD,Porcentaje de SD
6,hora,4,0.016817
8,direccion_normalizada,10815,45.469834
9,comuna,846,3.556864
10,tipo_calle,11045,46.436830
11,otra_direccion,18295,76.918226
15,geocodificacion_CABA,1213,5.099853
16,longitud,1209,5.083036
17,latutid,1209,5.083036
18,victima,10733,45.125079
19,acusado,15288,64.275804


In [11]:
# Reemplazamos "SD" por NaN en todo el DataFrame
df_hechos.replace(["SD","sd"], np.nan, inplace=True)

Preparamos los datos que usaremos al llamar a la función "data_cleaning".

In [12]:
# Columnas que pasaremos a minúcula
columns_to_lower = ["tipo_calle", "victima", "acusado", "gravedad"]

# Columnas que eliminaremos
columns_to_drop = ["aaaa", "mm", "dd", "hora", "direccion_normalizada", "otra_direccion", "calle", "altura", "cruce",
                   "participantes", "geocodificacion_CABA", "moto", "auto", "transporte_publico", "camion", "ciclista"]

# Columnas en las que reemplazaremos valores nulos por otros valores
fill_na_dict = {"gravedad":"leve"}

# Columnas que renombraremos
rename_dict = {"id":"id_siniestro", "n_victimas":"nro_victimas",
               "latutid":"latitud", "victima":"vehiculo_victima", "acusado":"vehiculo_acusado"}

# Columnas que convertiremos a entero
columns_to_int = ["nro_victimas", "franja_hora", "comuna"]

# Columnas que convertiremos a flotante
columns_to_float = ["latitud", "longitud"]

# Columnas que categorizaremos
columns_to_categorize = ["franja_hora", "tipo_calle", "comuna", "vehiculo_victima", "vehiculo_acusado", "gravedad"]

In [13]:
# Llamamos a la función "data_cleaning"
df_hechos_cleaned = data_cleaning(df_hechos,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    fill_na=fill_na_dict, # Rellenar nulos
                                    rename_columns=rename_dict,  # Renombrar columnas                                        
                                    convert_to_float=columns_to_float, # Convertir a float
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    categorize_columns=columns_to_categorize # Categorizar columnas
                                   )

Se pudo obervar la existencia de un valor igual a 0 en la columna "nro_victimas".

In [14]:
registros_nro_victimas_0 = df_hechos_cleaned[df_hechos_cleaned["nro_victimas"] == 0]
registros_nro_victimas_0

,id_siniestro,nro_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latitud,vehiculo_victima,vehiculo_acusado,gravedad
9928,PNA-2019-0005246,0,2019-12-20 00:00:00,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,leve


Al notar que la mayoría de columnas contienen valor NaN, se optó por consultar la segunda hoja del archivo "VICTIMAS" y se encontró que el registro corresponde a una persona, por lo que se reemplazará el valor 0 por 1.

In [15]:
# Reemplazamos valor en columnas
values_to_replace = {"comuna": {"No Especificada": np.nan}, "nro_victimas": {0: 1}}

In [16]:
# Llamamos nuevamente a la función "data_cleaning" para reemplar el valor
df_hechos_cleaned = data_cleaning(df_hechos_cleaned, replace_values=values_to_replace) # Reemplazar valores

In [17]:
# Observamos el resultado
df_hechos_cleaned.head(5)

,id_siniestro,nro_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latitud,vehiculo_victima,vehiculo_acusado,gravedad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9,14,NaN,-58.408911,-34.559658,ciclista,NaN,leve
1,LC-2019-0000053,1,2019-01-01 00:00:00,1,8,NaN,-58.443510,-34.669125,auto,NaN,leve
2,LC-2019-0000063,1,2019-01-01 00:00:00,2,8,NaN,-58.468335,-34.677556,NaN,NaN,leve
3,LC-2019-0000079,1,2019-01-01 00:00:00,2,7,NaN,-58.437425,-34.647349,peaton,NaN,leve
4,LC-2019-0000082,4,2019-01-01 00:00:00,4,3,NaN,-58.398225,-34.604579,auto,NaN,leve


#### 3.2. Dataset "VICTIMAS"

In [18]:
df_victimas = datos_lesiones["VICTIMAS"]

In [19]:
# Mostramos las primeras 5 filas del DataFrame
df_victimas.head()

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,SD,SD
3,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,45,SD
4,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,45,SD


In [20]:
# Revisamos las información general del DataFrame
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


Tras verificar el DataFrame, identificamos que los valores "SD" corresponden a "Sin Dato", por lo que se tomó la decisión de cambiar estos valores a NaN para estandarizar.

In [21]:
# Observamos la presencia de datos "SD" en el DataFrame
df_victimas.eq("SD").sum()

ID hecho                0
AAA                     0
MM                      0
DD                      0
FECHA                   0
VEHICULO_VICTIMA     4479
SEXO                 1929
EDAD_VICTIMA         3990
GRAVEDAD            20722
dtype: int64

In [22]:
# Llamamos a la función "analizar_valores_sd"
analizar_valores_sd(df_victimas)

,Columna,Cantidad de SD,Porcentaje de SD
5,VEHICULO_VICTIMA,4479,16.225321
6,SEXO,1929,6.987865
7,EDAD_VICTIMA,3990,14.453903
8,GRAVEDAD,20722,75.066111


In [23]:
# Reemplazamos "SD" por NaN en todo el DataFrame
df_victimas.replace(["SD","sd"], np.nan, inplace=True)

Preparamos los datos que usaremos al llamar a la función "data_cleaning".

In [25]:
# Columnas que pasaremos a minúcula
columns_to_lower = ["SEXO"]

# Columnas que eliminaremos
columns_to_drop = ["FECHA ", "AAA", "MM", "DD", "VEHICULO_VICTIMA", "GRAVEDAD"]

# Columnas que renombraremos
rename_dict = {"ID hecho":"id_siniestro", "SEXO":"sexo", "EDAD_VICTIMA":"edad"}

# Valores que reemplazaremos en columnas
values_to_replace = {"sexo": {"varon":"masculino", "mujer":"femenino"}}

# Columnas que categorizaremos
columns_to_categorize = ["sexo", "edad"]

# Columnas que agregaremos
new_columns_dict = {"rol": ""}

In [26]:
# Llamamos a la función "data_cleaning"
df_victimas_cleaned = data_cleaning(df_victimas,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas 
                                    replace_values=values_to_replace, # Reemplazar valores
                                    categorize_columns=columns_to_categorize, # Categorizar columnas
                                    new_columns=new_columns_dict, # Agregar columnas
                                    )

In [27]:
# Mostramos el resultado
df_victimas_cleaned.head()

,id_siniestro,sexo,edad,rol
0,LC-2019-0000053,masculino,57,
1,LC-2019-0000063,NaN,NaN,
2,LC-2019-0000079,masculino,NaN,
3,LC-2019-0000082,masculino,45,
4,LC-2019-0000082,femenino,45,


### 4. Combinación de Datos

In [28]:
# Fusionamos los DataFrames
lesiones = pd.merge(df_hechos_cleaned, df_victimas_cleaned, left_on="id_siniestro", right_on="id_siniestro", how="inner")

In [29]:
# Reordenamos las columnas del DataFrame
nuevo_orden = ['id_siniestro', 'nro_victimas', 'fecha', 'franja_hora', 'tipo_calle', 'comuna', 'longitud', 'latitud', 
           'vehiculo_victima', 'vehiculo_acusado', 'rol', 'sexo', 'edad', 'gravedad']

lesiones = lesiones[nuevo_orden]

In [30]:
# Mostramos el resultado
lesiones.head(5)

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado,rol,sexo,edad,gravedad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9,NaN,14,-58.408911,-34.559658,ciclista,NaN,,NaN,NaN,leve
1,LC-2019-0000053,1,2019-01-01 00:00:00,1,NaN,8,-58.443510,-34.669125,auto,NaN,,masculino,57,leve
2,LC-2019-0000063,1,2019-01-01 00:00:00,2,NaN,8,-58.468335,-34.677556,NaN,NaN,,NaN,NaN,leve
3,LC-2019-0000079,1,2019-01-01 00:00:00,2,NaN,7,-58.437425,-34.647349,peaton,NaN,,masculino,NaN,leve
4,LC-2019-0000082,4,2019-01-01 00:00:00,4,NaN,3,-58.398225,-34.604579,auto,NaN,,masculino,45,leve


### 5. Guardar el conjunto de datos limpio

In [31]:
# Los archivos se almacenan en local 
lesiones.to_csv("lesiones_cleaned.csv", index=False)